# 📊 Fairness Framework - Real-World Case Studies

**Duration**: 30-45 minutes

This notebook demonstrates the application of the Fairness Framework to real-world datasets:
1. **COMPAS** - Recidivism prediction
2. **Adult Income** - Income classification
3. **German Credit** - Credit risk assessment
4. **Bank Marketing** - Marketing campaign outcomes

## What you'll learn:
- How to analyze real-world datasets for bias
- How to check regulatory compliance (EEOC/ECOA)
- How to interpret results in practical contexts
- How to compare fairness across different use cases

## Setup

In [ ]:
# Add parent directory to path
import sys
sys.path.insert(0, '../..')

# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Fairness framework
from src.fairness_detector import FairnessDetector
from src.metrics import compute_all_metrics, is_fair
from src.visualization import plot_fairness_report, plot_group_comparison, create_fairness_dashboard
from src.utils import group_statistics

# Display settings
pd.set_option('display.max_columns', None)
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

print("✓ Setup complete!")

## Case Study 1: COMPAS - Recidivism Prediction

**Context**: COMPAS is a tool used in the US criminal justice system to predict recidivism risk.

**Known Issues**: ProPublica investigation found racial bias in predictions.

**Protected Attributes**: Race, Sex, Age

In [ ]:
# Note: This is a placeholder. Actual COMPAS data should be loaded from data/case_studies/compas/
# For demonstration, we'll create synthetic data with known bias patterns

print("📁 Loading COMPAS dataset...")
print("⚠️  Note: Using synthetic data for demonstration")
print("   Real COMPAS data should be placed in: data/case_studies/compas/")

# Create synthetic COMPAS-like data
np.random.seed(42)
n_samples = 1000

compas_data = pd.DataFrame({
    'age': np.random.randint(18, 70, n_samples),
    'race': np.random.choice(['African-American', 'Caucasian', 'Hispanic'], n_samples, p=[0.5, 0.3, 0.2]),
    'sex': np.random.choice(['Male', 'Female'], n_samples, p=[0.7, 0.3]),
    'priors_count': np.random.poisson(2, n_samples),
    'risk_score': np.random.randint(1, 11, n_samples),
})

# Create biased predictions (higher risk scores for African-American)
compas_data['high_risk'] = (compas_data['risk_score'] >= 5).astype(int)
# Add bias
mask = compas_data['race'] == 'African-American'
compas_data.loc[mask & (np.random.rand(n_samples) < 0.3), 'high_risk'] = 1

print(f"✓ Loaded {len(compas_data)} records")
print(f"\nRace distribution:")
print(compas_data['race'].value_counts())
print(f"\nHigh risk rate by race:")
print(compas_data.groupby('race')['high_risk'].mean())

In [ ]:
# Analyze COMPAS for bias
print("🔍 Analyzing COMPAS dataset for racial bias...\n")

detector = FairnessDetector(threshold=0.1, verbose=False)
detector.load_data(
    compas_data,
    target='high_risk',
    sensitive_attrs=['race']
)

results = detector.detect_bias()
print(results.summary())

In [ ]:
# Check EEOC compliance for COMPAS
print("⚖️  Checking EEOC Compliance...\n")

compliance = detector.check_eeoc_compliance()

print(f"Compliant: {compliance['compliant']}")
print(f"Impact Ratio: {compliance['impact_ratio']:.3f} (threshold: 0.80)")
print(f"\nSelection Rates by Race:")
for group, rate in compliance['selection_rates'].items():
    print(f"  {group}: {rate:.3f}")

if not compliance['compliant']:
    print(f"\n⚠️  Failing Groups: {', '.join(compliance['failing_groups'])}")

In [ ]:
# Visualize COMPAS results
plot_group_comparison(
    y_pred=compas_data['high_risk'],
    sensitive_attr=compas_data['race'],
    group_names=compas_data['race'].unique()
)

## Case Study 2: Adult Income Dataset

**Context**: Predict whether income exceeds $50K/year based on census data.

**Protected Attributes**: Sex, Race

**Use Case**: Employment screening, lending decisions

In [ ]:
print("📁 Loading Adult Income dataset...")
print("⚠️  Note: Using synthetic data for demonstration")
print("   Real Adult dataset should be placed in: data/case_studies/adult/")

# Create synthetic Adult-like data
n_samples = 1500
adult_data = pd.DataFrame({
    'age': np.random.randint(17, 90, n_samples),
    'education': np.random.randint(1, 17, n_samples),
    'hours_per_week': np.random.randint(1, 99, n_samples),
    'sex': np.random.choice(['Male', 'Female'], n_samples, p=[0.67, 0.33]),
    'race': np.random.choice(['White', 'Black', 'Asian', 'Other'], n_samples, p=[0.75, 0.12, 0.08, 0.05]),
})

# Create income prediction with gender bias
adult_data['high_income'] = (
    (adult_data['education'] > 10) & 
    (adult_data['hours_per_week'] > 35)
).astype(int)

# Add gender bias
mask = adult_data['sex'] == 'Female'
adult_data.loc[mask & (np.random.rand(n_samples) < 0.25), 'high_income'] = 0

print(f"✓ Loaded {len(adult_data)} records")
print(f"\nHigh income rate by sex:")
print(adult_data.groupby('sex')['high_income'].mean())

In [ ]:
# Analyze Adult dataset
print("🔍 Analyzing Adult Income dataset for gender bias...\n")

detector_adult = FairnessDetector(threshold=0.1)
detector_adult.load_data(
    adult_data,
    target='high_income',
    sensitive_attrs=['sex']
)

results_adult = detector_adult.detect_bias()
print(results_adult.summary())

## Case Study 3: German Credit Risk

**Context**: Assess credit risk for loan applicants.

**Protected Attributes**: Sex, Age

**Regulation**: ECOA compliance required

In [ ]:
print("📁 Loading German Credit dataset...")
print("⚠️  Note: Using synthetic data for demonstration")

# Create synthetic German Credit-like data
n_samples = 800
credit_data = pd.DataFrame({
    'age': np.random.randint(19, 75, n_samples),
    'sex': np.random.choice(['male', 'female'], n_samples, p=[0.7, 0.3]),
    'credit_amount': np.random.randint(250, 18500, n_samples),
    'duration': np.random.randint(4, 72, n_samples),
    'credit_history': np.random.choice(['good', 'poor', 'critical'], n_samples, p=[0.6, 0.3, 0.1]),
})

# Create credit approval with age bias
credit_data['approved'] = (
    (credit_data['credit_history'] == 'good') & 
    (credit_data['credit_amount'] < 10000)
).astype(int)

# Add age bias (discriminate against younger applicants)
mask = credit_data['age'] < 25
credit_data.loc[mask & (np.random.rand(n_samples) < 0.3), 'approved'] = 0

# Create age groups for analysis
credit_data['age_group'] = pd.cut(credit_data['age'], bins=[0, 25, 60, 100], labels=['young', 'middle', 'senior'])

print(f"✓ Loaded {len(credit_data)} records")
print(f"\nApproval rate by age group:")
print(credit_data.groupby('age_group')['approved'].mean())

In [ ]:
# Analyze German Credit for ECOA compliance
print("🔍 Analyzing German Credit dataset...\n")

detector_credit = FairnessDetector(threshold=0.1)
detector_credit.load_data(
    credit_data,
    target='approved',
    sensitive_attrs=['age_group']
)

# Check ECOA compliance
ecoa_compliance = detector_credit.check_ecoa_compliance()

print("⚖️  ECOA Compliance Check:")
print(f"\nCompliant: {ecoa_compliance['compliant']}")
print(f"Disparate Impact: {ecoa_compliance['disparate_impact']:.3f}")
print(f"\nApproval Rates:")
for group, rate in ecoa_compliance['approval_rates'].items():
    print(f"  {group}: {rate:.3f}")

if ecoa_compliance['disadvantaged_groups']:
    print(f"\n⚠️  Disadvantaged Groups: {', '.join(ecoa_compliance['disadvantaged_groups'])}")

## Comparative Analysis

Let's compare fairness across all case studies.

In [ ]:
# Compute metrics for all datasets
print("📊 Computing fairness metrics for all case studies...\n")

# COMPAS metrics
compas_metrics = compute_all_metrics(
    compas_data['high_risk'],
    compas_data['high_risk'],
    compas_data['race']
)

# Adult metrics
adult_metrics = compute_all_metrics(
    adult_data['high_income'],
    adult_data['high_income'],
    adult_data['sex']
)

# Credit metrics
credit_metrics = compute_all_metrics(
    credit_data['approved'],
    credit_data['approved'],
    credit_data['age_group']
)

# Create comparison DataFrame
comparison = pd.DataFrame({
    'COMPAS (Race)': compas_metrics,
    'Adult (Sex)': adult_metrics,
    'Credit (Age)': credit_metrics
})

print("Fairness Metrics Comparison:")
print("=" * 70)
print(comparison.round(3))
print("=" * 70)

# Check which ones are fair
print("\nFairness Assessment (threshold: 0.1):")
for metric in comparison.index:
    print(f"\n{metric}:")
    for dataset in comparison.columns:
        value = comparison.loc[metric, dataset]
        fair = is_fair(value, metric)
        status = "✓ FAIR" if fair else "✗ BIASED"
        print(f"  {dataset:20s}: {value:7.3f}  [{status}]")

## Key Takeaways

### 1. Different Contexts, Different Biases
- **COMPAS**: Racial bias in criminal justice
- **Adult Income**: Gender bias in income prediction
- **German Credit**: Age discrimination in lending

### 2. Regulatory Considerations
- **EEOC**: Applies to employment decisions (Adult Income)
- **ECOA**: Applies to credit decisions (German Credit)
- **Criminal Justice**: Different standards (COMPAS)

### 3. Mitigation Strategies
1. **Data Collection**: Ensure representative sampling
2. **Feature Engineering**: Remove or transform biased features
3. **Algorithmic Adjustments**: Apply fairness constraints
4. **Post-processing**: Adjust predictions to achieve fairness
5. **Ongoing Monitoring**: Regular audits and updates

### 4. Trade-offs
- Fairness vs. Accuracy
- Different fairness metrics may conflict
- Context-specific considerations

## Next Steps

1. **Load Real Data**: Replace synthetic data with actual case study datasets
2. **Try Mitigation**: Experiment with bias mitigation techniques
3. **Compare Tools**: Benchmark against AIF360, Fairlearn
4. **Domain Expertise**: Consult with domain experts for interpretation
5. **Continuous Monitoring**: Implement ongoing fairness audits

## Resources

- **COMPAS Analysis**: ProPublica (2016)
- **Adult Dataset**: UCI ML Repository
- **German Credit**: UCI ML Repository
- **Regulations**: EEOC, ECOA, GDPR, EU AI Act
- **Tools**: AIF360, Fairlearn, Aequitas

---

**Remember**: Fairness is context-dependent and requires domain expertise. Always consult with stakeholders and legal experts when making decisions based on fairness analyses.